In [1]:
#load in dependencies
import pandas as pd
import numpy as np
import bz2
import json

In [2]:
#files are saved by month and year as a list of JSON's compressed into a bz2 file
#for memory saving purposes I ran this for each year separately, and eventually for month as well
#future versions of this code should make everything else a def, then run this cell
years = ["2010"]
#years = ["2006", "2007", "2008", "2009", "2010", "2011"]
months = ["01"]#, "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
uncleaned_df = pd.DataFrame()

for y in years:
    for m in months:
        try:
            filename = "data/" + y + "/RC_" + y + "-" + m + ".bz2"
            parsed_file = pd.read_json(filename, compression='bz2', lines=True)
            parsed_file['month'] = m
            parsed_file['year'] = y
            uncleaned_df = uncleaned_df.append(parsed_file)
            print(filename + " parsed")
            
        except:
            print("file named " + filename + " does not exist")

data/2010/RC_2010-01.bz2 parsed


In [ ]:
#check out layout of each post
print(uncleaned_df.head(2))

In [3]:
#clean the data
clean_df = uncleaned_df[['id', 'score', 'controversiality', 'subreddit', 'body', 'month', 'year']]
print(clean_df.dtypes)
print(clean_df.isnull().sum(axis=0)) #check for null values

id                  object
score                int64
controversiality     int64
subreddit           object
body                object
month               object
year                object
dtype: object
id                  0
score               0
controversiality    0
subreddit           0
body                0
month               0
year                0
dtype: int64


In [4]:
#sort into xbox and ps mentions, ignoring posts that have neither
ps_list = ['playstation', 'ps1', 'ps2', 'ps3', 'ps4', 'ps5']
xbox_list = ['xbox']
blacklist = ['Fluxbox', 'fluxbox', 'ps2pdf', 'cps3', 'mps3', 'Trixbox', 'rps25', 'aps2', 'gps2', 'lps3', 'pps25']

#check the body of each row in clean_df for elements of ps_list or xbox_list
#if it contains ps, make ps column true
#if it contains xbox, make xbox column true
#if either of the above pass, they go into the new dataframe df
def ps_mention(row):
    mention = False
    if not any(entry in row for entry in blacklist):
        if any(x in row for x in ps_list):
            mention = True
    return mention

def xbox_mention(row):
    mention = False
    if not any(entry in row for entry in blacklist):
        if any(x in row for x in xbox_list):
            mention = True
    return mention

clean_df['PS'] = clean_df['body'].apply(ps_mention)
clean_df['XBOX'] = clean_df['body'].apply(xbox_mention)

df = clean_df[(clean_df['PS'] == True) | (clean_df['XBOX'] == True)]
df.set_index('id', inplace=True, drop=True)

<ipython-input-4-77d2b50b9069>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['PS'] = clean_df['body'].apply(ps_mention)
<ipython-input-4-77d2b50b9069>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['XBOX'] = clean_df['body'].apply(xbox_mention)


In [5]:
print(clean_df.shape)
print(df.shape)

(2884096, 9)
(2193, 8)


In [6]:
#save dataframes as files for future use (done one year at a time, otherwise this needs to be in a loop)
df.to_json('dataframes/df'+years[0]+months[0]+'.json')

#df2 = pd.read_json('dataframes/df'+years[0]+'.json')
#print(df2.head())

In [ ]:
m = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

final_df = pd.DataFrame()
for x in m:
    loaded_df = pd.read_json('dataframes/df'+years[0]+x+'.json')
    final_df = final_df.append(loaded_df)
final_df.to_json('dataframes/df'+years[0]+'.json')